In [1]:
from nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='./data/nuScenes', verbose=True)
# path = nusc.render_sample_data('73cd8a8a6c79453f9a7236d550de1e7a')


Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 22.1 seconds.
Reverse indexing ...
Done reverse indexing in 5.1 seconds.


In [ ]:
# Loop through all sample_data and verify if LIDAR_TOP file exists
import numpy as np
from tqdm import tqdm
for sample in nusc.sample:
    lidar_token = sample['data']['LIDAR_TOP']
    lidar_path = nusc.get_sample_data_path(lidar_token)
    # try to open the file
    try:
        # if 'n015-2018-07-18-11-07-57+0800__LIDAR_TOP__1531883530900022' in lidar_path:
        if 'sweeps' in lidar_path:
            print(lidar_path)
        # print(lidar_path)
        ptcloud = np.fromfile(lidar_path)
    except:
        print(f'Error: {lidar_token}, {lidar_path} does not exist')
        continue

In [18]:
import pickle

with open('/home/vxm240030/CenterPoint/data/nuScenes/full/infos_train_10sweeps_withvelo_filter_True.pkl', 'rb') as f:
    data = pickle.load(f)

In [26]:
count = 0
import os
import numpy as np
for datai in data:
    path = datai['lidar_path']
    if not os.path.exists(path):
        print(path)
        np.fromfile(datai)
        count += 1
